In [1]:
import gym
import numpy as np
from numpy import random
env = gym.make('Blackjack-v0')

[2020-04-28 15:38:32,640] Making new env: Blackjack-v0
/home/harsh/anaconda3/envs/DEEPRL/lib/python3.5/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [2]:
def strategy(state):
    i,j,k = state
    if Q[i,j,k,0].any()>Q[i,j,k,1].any():
        res = 0
    else:
        res = 1
    return res 
def generate_episode(episode,done):
    state,action,reward = episode[0]
    while done is False:
        state, reward, done, info = env.step(action)
        episode.append(tupletolist(state))
        action = strategy(state)
        episode.append(action)
        episode.append(reward) 
    return episode
def generate_episode2(state):
    episode=[]
    done = False
    while done is False:
        episode.append(tupletolist(state))
        action = strategy(state)
        episode.append(action)
        state, reward, done, info = env.step(action)
        episode.append(reward) 
    return episode
def update_action_value_and_pi(episode):
    for s in range(len(episode)-3,-1,-3):
        s0,s1,s2,ac,ret = episode[s][0],episode[s][1],episode[s][2],episode[s+1], episode[s+2]
        Q[s0,s1,s2,ac] += 1.0*(ret- Q[s0,s1,s2,ac])/count_n[s0,s1,s2,ac]
        count_n[s0,s1,s2,ac]+=1
        if Q[s0,s1,s2,0].any()>=Q[s0,s1,s2,1].any():
            pi[s0,s1,s2] = 0
        else: 
            pi[s0,s1,s2] = 1 
def tupletolist(state):
    res=[state[0],state[1]]
    if(state[2]==False):
        res.append(0)
    else:
        res.append(1)
    return res

In [3]:
def train():
    for i in range(1,33):
        for j in range(1,12):
            for k in [False,True]:
                for act in range(2):
                    for itr in range(2):
                        G=0
                        episode = []
                        state = (i,j,k)
                        episode.append(tupletolist(state))
                        state, reward, done, info = env.step(act)
                        episode.append(act)
                        episode.append(reward)
                        episode = generate_episode(episode,done)
                        update_action_value_and_pi(episode)                                    
    for itr in range(500000):
        episode = []
        state = env.reset()
        episode = generate_episode2(state)
        update_action_value_and_pi(episode)

In [4]:
#initialize
pi = random.randint(2, size=(33,12,2))
Q = np.zeros((33,12,2,2))
for i in range(33):
    for j in range(12):
        for k in range(2):
            for l in range(2):
                Q[i,j,k,l] = random.rand()*2
count_n = np.ones((33,12,2,2))  # see exercise 5.4
train()

In [5]:
def plot_policy(policy):

    def get_Z(x, y, usable_ace):
        if (x,y,usable_ace) in policy:
            return policy[x,y,usable_ace]
        else:
            return 1

    def get_figure(usable_ace, ax):
        x_range = np.arange(11, 22)
        y_range = np.arange(10, 0, -1)
        X, Y = np.meshgrid(x_range, y_range)
        Z = np.array([[get_Z(x,y,usable_ace) for x in x_range] for y in y_range])
        surf = ax.imshow(Z, cmap=plt.get_cmap('Pastel2', 2), vmin=0, vmax=1, extent=[10.5, 21.5, 0.5, 10.5])
        plt.xticks(x_range)
        plt.yticks(y_range)
        plt.gca().invert_yaxis()
        ax.set_xlabel('Player\'s Current Sum')
        ax.set_ylabel('Dealer\'s Showing Card')
        ax.grid(color='w', linestyle='-', linewidth=1)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cbar = plt.colorbar(surf, ticks=[0,1], cax=cax)
        cbar.ax.set_yticklabels(['0 (STICK)','1 (HIT)'])
            
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(121)
    ax.set_title('Usable Ace')
    get_figure(True, ax)
    ax = fig.add_subplot(122)
    ax.set_title('No Usable Ace')
    get_figure(False, ax)
    plt.show()

In [7]:
usable_ace = pi[:,:,1]

In [8]:
usable_ace

array([[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,